## Напоминание do-нотация

In [1]:
x = Just 3
y = Just 4
-- z = x + y

z = do
      a <- x  -- как будто беру значение изнутри x
      b <- y
      return $ a + b -- "возвращаю" значение, завернутое в монаду

z

Just 7

На самом деле do-нотация это синтаксический сахар:

In [7]:
x = Just 3
y = Just 4
-- z = x + y
-- заменяем  a <- x на x >>= (\a -> )
z = x >>= (\a -> 
    -- заменяем  b <- y на y >>= (\a -> )
             y >>= (\b -> return $ a + b)
    ) 

z

Just 7

Вспомним функцию `x >>= f`, она берет "значение внутри монады" `x`, и применяет к нему функцию `f`. Функция `f` должна сама возвращать значение в монаде.

Попробуем написать эту функцию для `Maybe`

In [8]:
(>>=*) :: Maybe a -> (a -> Maybe b) -> Maybe b
Nothing >>=* _ = Nothing
(Just x) >>=* f = f x

x = Just 3
y = Just 4
-- z = x + y
-- заменяем  a <- x на x >>= (\a -> )
z = x >>=* (\a -> 
    -- заменяем  b <- y на y >>= (\a -> )
             y >>=* (\b -> return $ a + b)
    ) 

z

Just 7

## IO

Тип `IO a` означает значение типа `a`, которое в данный момент недоступно, но будет доступно после выполнения некоторых операций ввода/вывода. (чтение с клавиатуры, из файла)

Часто операции ввода/вывода не возвращают интересных значений, например, так устроен `print`. Он имеет побочный эффект — печать на экране, но нам не нужен результат. Тогда в качестве типа a используется тип Unit, фактически это кортеж из 0 элементов: `()`
`(a, b)`, `(a, b, c)`

Тип `()` содержит всего одно значение, оно тоже записывается `()`.
(сравните тип `(Int, String)` и значение `(42, "hello")`.

Функции типа `print`, и функция `Main` часто имеют тип `IO ()`.

In [11]:
z = do
       x <- readLn 
       y <- readLn
       return $ x + y

-- z = do {x <- readLn ; y <- readLn ; return $ x + y}

z

: 

Попробуем прочитать текст из файла:

In [16]:
import System.IO

readHello :: IO String
readHello = withFile "a.txt" ReadMode (\h -> 
             do
              text <- hGetContents h
              return $ "file contents: " ++ text
             )

readHello

: 

In [32]:
import System.IO

readHello :: IO String
readHello = do
               handle <- openFile "a.txt" ReadMode
               putStr "прочитаем содержимое файла\n"
               text <- hGetContents handle
               putStr "закроем файл\n"
               -- hClose handle
               return $ "file contents: " ++ text
               
do
   text <- readHello
   putStr text -- putStr только для строк
   -- putStr 42 -- нельзя
   print 42 -- сначала вызывает show, потом putStr

: 

Как в do нотации работают строки без `<-`, например, что происходит в этом случае:

In [34]:
do
   putStr "hello\n"
   putStr "world\n"

hello
world

Это преобразуется в

In [36]:
putStr "hello\n" >>= (\_ -> 
   putStr "world\n")

Line 1: Use >>
Found:
putStr "hello\n" >>= (\ _ -> putStr "world\n")
Why not:
putStr "hello\n" >> putStr "world\n"

hello
world

На самом деле для опрерации `x >>= (\_ -> y)` есть операция `x >> y`. Получается:

In [37]:
putStr "hello\n" >> putStr "world\n" 

hello
world

## Сделаем самостоятельно монаду Random

Монада `Random a` будет содержать случайное значение типа `a`. В принципе, такой тип уже есть в Haskell, но он завязан на IO.

Будем генерировать случайные числа с помощью «линейный конгруэнтный генератор». Идея такова, что есть текущее случайное значение, и мы можем на его основе вычислить следующее значение:

$x_n = (ax_{n-1} + c) \mod m$

При этом $a$, $c$, $m$ фиксированы, $x_0$ задаётся пользователем.

Можно ли в Haskell сделать генератор случайных чисел такого типа?

In [ ]:
let gen = createGenerator
let a = generate gen
    b = generate gen in a + b

В этом коде результат для a и b будет одинаковый, потому что в Haskell у функций нет подочных эффектов, вызов с одними аргументами приводит к одному результату.

На самом деле, генератор случайных чисел должен после генерации возвращать новый генератор. Т.е. генерация случайного числа должна выглядеть примерно так:

In [ ]:
let (a, gen2) = generate gen,
    (b, gen3) = generate gen2 in a + b

Введем, наконец, типы и операции

In [12]:
data Random a = Random Integer (Integer -> a)

instance Show (Random a) where
  show (Random x _) = "Random[" ++ show x ++ "]"

generate :: Random a -> (a, Random a)
generate (Random x f) = (f x, Random nextX f) where
                            nextX = mod (134775813*x + 1) (2^32)
                            
-- сделаем генератор чисел в диапазоне от 0 до 2^32
randomInts x0 = Random x0 id

randomIntsInRange min max x0 = Random x0 (\x -> min + (x `mod` (max - min + 1)))
-- число в диапазоне от 1 до 2: 1 + x `mod` 2

Пробуем использовать этот генератор:

In [15]:
let gen = randomIntsInRange 1 10 2384
let (x, gen2) = generate gen
let (y, gen3) = generate gen2

x
y
x + y
gen
gen2
gen3

5

10

15

Random[2384]

Random[3477958289]

Random[4128107734]

Сделаем теперь так, чтобы `Random a` стал функтором. Т.е. к случайным значениям, которые генерируются внурти генератора, можно будет применить функцию. Должно это работать так:

Например, если `gen1 :: Random Int` генерирует числа 0 или 1, то `fmap gen1 (\x -> if x == 0 then "орёл" else "решка")` генерирует орлов и решек:

In [18]:
instance Functor Random where
    -- fmap :: (Random a) -> (a -> b) -> Random b
    -- fmap (Random x0 f) fun = Random x0 (\x -> fun (f x))
    fmap fun (Random x0 f) = Random x0 (fun . f)

Пробуем с орлом и решкой

In [22]:
gen01 = randomIntsInRange 0 1 283476
genHeadTail = (\x -> if x == 0 then "head" else "tail") <$> gen01 

let (x, gen01_1) = generate gen01
let (y, gen01_2) = generate gen01_1
let (z, gen01_3) = generate gen01_2

let (a, genHeadTail1) = generate genHeadTail
let (b, genHeadTail2) = generate genHeadTail1
let (c, genHeadTail3) = generate genHeadTail2

x
y
z
a
b
c

0

1

0

"head"

"tail"

"head"